# Notebook: Transform dữ liệu từ tầng Bronze lên tầng Silver

## Mục tiêu
Transform dữ liệu customer từ tầng Bronze (raw JSON) lên tầng Silver (structured data) với các bước:
1. **Extract**: Parse JSON thành tabular format
2. **Transform**: Làm sạch và chuẩn hóa dữ liệu  
3. **Load**: Lưu vào Silver layer với schema tối ưu
4. **Document**: Tạo Data Dictionary

## Quy trình 8 bước chuẩn
1. **Import & Connect DB** - Thiết lập môi trường
2. **Load dữ liệu từ Bronze** - Đọc raw data
3. **JSON Structure Analysis** - Phân tích cấu trúc JSON
4. **Parse & Flatten JSON** - Chuyển đổi sang tabular
5. **Data Quality Check** - Đánh giá chất lượng dữ liệu
6. **Data Cleaning** - Làm sạch và chuẩn hóa
7. **Load vào Silver** - Lưu với schema tối ưu
8. **Data Dictionary** - Tạo tài liệu metadata

---


## Bước 1: Import & Connect DB

### Mục tiêu
- Import các thư viện cần thiết
- Load biến môi trường từ file `.env`
- Tạo kết nối tới MySQL database

### Kết nối Database
- `bronze_engine` → Đọc dữ liệu gốc từ schema **Bronze**
- `silver_engine` → Ghi dữ liệu đã làm sạch sang schema **Silver**

> **Lưu ý**: Đây là bước khởi tạo, chưa có quyết định gì, chỉ thiết lập môi trường làm việc.


In [1]:
import os
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")

# Tạo kết nối tới Bronze và Silver database
bronze_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}")
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")

print("✅ Đã kết nối Bronze và Silver database thành công")
print(f"📊 Bronze DB: {DB_BRONZE}")
print(f"📊 Silver DB: {DB_SILVER}")


✅ Đã kết nối Bronze và Silver database thành công
📊 Bronze DB: winner_bronze
📊 Silver DB: winner_silver


## Bước 2: Load dữ liệu từ Bronze

### Mục tiêu
- Lấy toàn bộ bảng `customers_raw` từ Bronze database
- Hiển thị thông tin cơ bản về dataset (shape, columns)
- Xem 1 record JSON mẫu để nắm cấu trúc dữ liệu

### Quy trình
1. **Load data**: Đọc từ `customers_raw` table
2. **Basic info**: Hiển thị shape và columns
3. **Sample JSON**: Parse và hiển thị 1 record mẫu

> **Decision Point**: Từ JSON mẫu, ta sẽ quan sát để xác định có những nhóm trường nào (định danh, hành vi, địa chỉ...) và quyết định sẽ extract những field nào.


In [2]:
# Load dữ liệu customers từ Bronze database
customers_df = pd.read_sql("SELECT * FROM customers_raw", bronze_engine)

# Hiển thị thông tin cơ bản về dataset
print("=== THÔNG TIN DATASET ===")
print(f"📊 Shape: {customers_df.shape}")
print(f"📋 Columns: {list(customers_df.columns)}")
print(f"💾 Memory usage: {customers_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị 1 record JSON mẫu để nắm cấu trúc
print("\n=== JSON STRUCTURE SAMPLE ===")
sample_json = json.loads(customers_df["raw_json"].iloc[0])
print(json.dumps(sample_json, indent=2, ensure_ascii=False))


=== THÔNG TIN DATASET ===
📊 Shape: (36090, 4)
📋 Columns: ['shop_id', 'customer_id', 'raw_json', 'extracted_at']
💾 Memory usage: 107.94 MB

=== JSON STRUCTURE SAMPLE ===
{
  "returned_order_count": 0,
  "is_block": false,
  "customer_id": "c7a583d6-392d-4308-a02b-67c001ee03db",
  "count_referrals": 0,
  "username": null,
  "emails": [],
  "inserted_at": "2025-08-16T02:19:59",
  "creator": null,
  "list_voucher": [],
  "name": "Thinh Bui",
  "assigned_user_id": null,
  "last_order_at": null,
  "date_of_birth": null,
  "id": "e906c5c7-ea19-42e5-a971-57cf09c94417",
  "fb_id": "377626778776391_31407011708897267",
  "notes": [],
  "total_amount_referred": null,
  "is_adjust_debts": null,
  "is_discount_by_level": true,
  "updated_at": "2025-08-16T02:19:59",
  "conversation_link": "https://pancake.vn/377626778776391?customer_id=c7a583d6-392d-4308-a02b-67c001ee03db",
  "order_count": 1,
  "order_sources": [
    "-1"
  ],
  "tags": [],
  "succeed_order_count": 0,
  "user_block_id": null,
  "pho

## Bước 3: JSON Structure Analysis

### Mục tiêu
- Scan toàn bộ keys trong JSON để hiểu đầy đủ cấu trúc
- Phân loại các trường theo mức độ quan trọng
- Chuẩn bị cho việc lựa chọn fields để extract

### Quy trình
1. **Scan keys**: Duyệt qua 200 records đầu để lấy tất cả keys
2. **Categorize**: Phân loại keys theo business value
3. **Decision**: Quyết định extract những field nào

> **Decision Point**: Từ danh sách keys, ta sẽ quyết định chọn những cột quan trọng để parse (ví dụ: customer_id, name, gender, order_count...) và bỏ các cột ít giá trị (notes, creator...).

### Phân loại trường theo mức độ quan trọng:
- **High Priority**: Định danh, thông tin cơ bản, metrics chính
- **Medium Priority**: Thông tin bổ sung, hành vi
- **Low Priority**: Metadata, logs, fields ít sử dụng


In [3]:
# Scan tất cả keys trong JSON để hiểu đầy đủ cấu trúc
print("=== SCANNING JSON KEYS ===")
all_keys = set()

# Scan 200 records đầu tiên để lấy tất cả keys có thể có
for raw in customers_df["raw_json"].head(200):
    d = json.loads(raw)
    all_keys.update(d.keys())

print(f"📊 Tổng số keys tìm thấy: {len(all_keys)}")
print(f"📋 Danh sách keys (sorted):")
for i, key in enumerate(sorted(all_keys), 1):
    print(f"{i:2d}. {key}")

# Phân tích cấu trúc nested
print(f"\n=== NESTED STRUCTURE ANALYSIS ===")
sample_data = json.loads(customers_df["raw_json"].iloc[0])

def analyze_nested_structure(data, prefix=""):
    """Phân tích cấu trúc nested objects và arrays"""
    structure = {}
    for key, value in data.items():
        if isinstance(value, dict):
            structure[f"{prefix}{key}"] = "object"
            structure.update(analyze_nested_structure(value, f"{prefix}{key}."))
        elif isinstance(value, list) and value:
            structure[f"{prefix}{key}"] = f"array[{len(value)}]"
            if value and isinstance(value[0], dict):
                structure.update(analyze_nested_structure(value[0], f"{prefix}{key}[0]."))
        else:
            structure[f"{prefix}{key}"] = type(value).__name__
    return structure

nested_structure = analyze_nested_structure(sample_data)
print(f"📊 Cấu trúc nested objects và arrays:")
for key, value_type in sorted(nested_structure.items()):
    if "." in key or "[" in key:  # Chỉ hiển thị nested structures
        print(f"  {key}: {value_type}")


=== SCANNING JSON KEYS ===
📊 Tổng số keys tìm thấy: 40
📋 Danh sách keys (sorted):
 1. active_levera_pay
 2. assigned_user_id
 3. conversation_link
 4. conversation_tags
 5. count_referrals
 6. creator
 7. creator_id
 8. currency
 9. current_debts
10. customer_id
11. date_of_birth
12. emails
13. fb_id
14. gender
15. id
16. inserted_at
17. is_adjust_debts
18. is_block
19. is_discount_by_level
20. last_order_at
21. level
22. list_voucher
23. name
24. notes
25. order_count
26. order_sources
27. phone_numbers
28. purchased_amount
29. referral_code
30. returned_order_count
31. reward_point
32. shop_customer_addresses
33. shop_id
34. succeed_order_count
35. tags
36. total_amount_referred
37. updated_at
38. used_reward_point
39. user_block_id
40. username

=== NESTED STRUCTURE ANALYSIS ===
📊 Cấu trúc nested objects và arrays:


## Bước 4: Parse & Flatten JSON

### Mục tiêu
- Định nghĩa hàm parse_customer để extract các trường đã chọn
- Flatten các trường nested như phone_numbers, emails, order_sources
- Parse nested objects như shop_customer_addresses
- Chuyển đổi từ JSON sang DataFrame tabular

### Quy trình
1. **Define parsing function**: Tạo hàm parse_customer với logic extract
2. **Handle arrays**: Chuyển arrays thành comma-separated strings
3. **Handle nested objects**: Extract fields từ nested objects
4. **Apply parsing**: Áp dụng hàm cho toàn bộ dataset

### Các trường được chọn (25 fields)
**Định danh & Cơ bản**: customer_id, id, name, shop_id
**Thông tin cá nhân**: gender, phone_numbers, emails, date_of_birth  
**Hành vi mua hàng**: order_count, succeed_order_count, returned_order_count, purchased_amount, last_order_at
**Thời gian**: inserted_at, updated_at
**Trạng thái**: is_block, is_discount_by_level, active_levera_pay
**Tài chính & Loyalty**: reward_point, used_reward_point, current_debts, count_referrals, total_amount_referred
**Marketing & Tracking**: referral_code, fb_id, order_sources, conversation_link
**Địa chỉ**: shop_customer_addresses (nested)
**Metadata**: currency

> **Decision Point**: Sau cell này, ta có bảng dạng sạch hơn để kiểm tra chất lượng dữ liệu.


In [4]:
def parse_customer(row):
    """
    Parse JSON customer data thành tabular format
    Extract 25 trường đã chọn với xử lý arrays và nested objects
    """
    d = json.loads(row["raw_json"])
    
    # Xử lý nested address object
    addr = d.get("shop_customer_addresses", [{}])[0] if d.get("shop_customer_addresses") else {}
    
    return {
        # Định danh & Cơ bản (4 fields)
        "customer_id": d.get("customer_id"),
        "id": d.get("id"), 
        "name": d.get("name"),
        "shop_id": d.get("shop_id"),
        
        # Thông tin cá nhân (4 fields)
        "gender": d.get("gender"),
        "phone": ",".join(d.get("phone_numbers", [])) if d.get("phone_numbers") else None,
        "email": ",".join(d.get("emails", [])) if d.get("emails") else None,
        "date_of_birth": d.get("date_of_birth"),
        
        # Hành vi mua hàng (5 fields)
        "order_count": d.get("order_count"),
        "succeed_order_count": d.get("succeed_order_count"),
        "returned_order_count": d.get("returned_order_count"),
        "purchased_amount": d.get("purchased_amount"),
        "last_order_at": d.get("last_order_at"),
        
        # Thời gian (2 fields)
        "inserted_at": d.get("inserted_at"),
        "updated_at": d.get("updated_at"),
        
        # Trạng thái (3 fields)
        "is_block": d.get("is_block"),
        "is_discount_by_level": d.get("is_discount_by_level"),
        "active_levera_pay": d.get("active_levera_pay"),
        
        # Tài chính & Loyalty (5 fields)
        "reward_point": d.get("reward_point"),
        "used_reward_point": d.get("used_reward_point"),
        "current_debts": d.get("current_debts"),
        "count_referrals": d.get("count_referrals"),
        "total_amount_referred": d.get("total_amount_referred"),
        
        # Marketing & Tracking (4 fields)
        "referral_code": d.get("referral_code"),
        "fb_id": d.get("fb_id"),
        "order_sources": ",".join(d.get("order_sources", [])) if d.get("order_sources") else None,
        "conversation_link": d.get("conversation_link"),
        
        # Địa chỉ (nested object - extract key fields)
        "address_province_id": addr.get("province_id"),
        "address_district_id": addr.get("district_id"),
        "address_commune_id": addr.get("commune_id"),
        "address_full": addr.get("full_address"),
        "address_postcode": addr.get("postcode"),
        
        # Metadata (1 field)
        "currency": d.get("currency")
    }

# Áp dụng parsing function cho toàn bộ dataset
print("=== PARSING JSON DATA ===")
customers_parsed = customers_df.apply(parse_customer, axis=1, result_type="expand")

# Hiển thị kết quả parsing
print(f"✅ Parsing completed successfully")
print(f"📊 Parsed shape: {customers_parsed.shape}")
print(f"📋 Columns extracted: {len(customers_parsed.columns)}")
print(f"💾 Memory usage: {customers_parsed.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print(f"\n=== SAMPLE PARSED DATA ===")
print(customers_parsed.head(3))


=== PARSING JSON DATA ===
✅ Parsing completed successfully
📊 Parsed shape: (36090, 33)
📋 Columns extracted: 33
💾 Memory usage: 48.24 MB

=== SAMPLE PARSED DATA ===
                            customer_id                                    id  \
0  c7a583d6-392d-4308-a02b-67c001ee03db  e906c5c7-ea19-42e5-a971-57cf09c94417   
1  590ba7de-e4fe-43b3-8013-27fc69937edf  6c5b40d2-ae42-4e13-8019-f3f9127d75a0   
2  a9e62388-57c2-4834-a87c-7c1025aea92d  765c531b-743f-4f30-b6b0-0d6f3579630f   

          name    shop_id gender       phone email date_of_birth  order_count  \
0    Thinh Bui  230361475   male  0903693389  None          None            1   
1  Truong Minh  230361475   male  0907809070  None          None            1   
2  Quach Quach  230361475   male  0986533988  None          None            1   

   succeed_order_count  ...  referral_code                              fb_id  \
0                    0  ...       GM5wdxsy  377626778776391_31407011708897267   
1                    0  

## Bước 5: Data Quality Check

### Mục tiêu
- Tính tỷ lệ null từng cột để quyết định giữ/lược bỏ
- Check duplicate theo customer_id
- In sample values cho các field quan trọng
- Đánh giá chất lượng dữ liệu tổng thể

### Quy trình
1. **Null analysis**: Tính tỷ lệ null từng cột
2. **Duplicate check**: Kiểm tra duplicate theo primary key
3. **Value analysis**: Phân tích giá trị của các trường quan trọng
4. **Quality assessment**: Đánh giá tổng thể chất lượng dữ liệu

### Decision Rules
- Nếu cột null > 70% → cân nhắc drop
- Nếu có duplicate → xác định rule xử lý (giữ bản mới nhất)
- Nếu có giá trị bất thường → quyết định chuẩn hóa mapping

### Trường quan trọng cần kiểm tra
- gender: Kiểm tra giá trị hợp lệ (male/female)
- is_block: Boolean values
- is_discount_by_level: Boolean values  
- active_levera_pay: Boolean values
- order_count, purchased_amount: Numeric values


In [5]:
# Data Quality Analysis
print("=== DATA QUALITY ANALYSIS ===")

# 1. Null ratio analysis
print("\n1. NULL RATIO ANALYSIS")
null_ratio = customers_parsed.isnull().mean().sort_values(ascending=False)
print("Tỷ lệ null theo cột (sorted):")
for col, ratio in null_ratio.items():
    print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%)")

# 2. Duplicate check
print(f"\n2. DUPLICATE CHECK")
dup_count = customers_parsed["customer_id"].duplicated().sum()
print(f"Số lượng customer_id trùng lặp: {dup_count}")

# Check duplicate theo id (internal ID)
dup_id_count = customers_parsed["id"].duplicated().sum()
print(f"Số lượng id (internal) trùng lặp: {dup_id_count}")

# 3. Value analysis cho các trường quan trọng
print(f"\n3. VALUE ANALYSIS")

# Gender analysis
print("\n--- GENDER ANALYSIS ---")
if 'gender' in customers_parsed.columns:
    gender_counts = customers_parsed['gender'].value_counts(dropna=False)
    print(gender_counts)
    
    # Check for unusual values
    valid_genders = ['male', 'female', 'Male', 'Female', 'M', 'F']
    invalid_genders = customers_parsed[~customers_parsed['gender'].isin(valid_genders)]['gender'].value_counts()
    if len(invalid_genders) > 0:
        print("Invalid gender values found:")
        print(invalid_genders)

# Boolean fields analysis
boolean_fields = ['is_block', 'is_discount_by_level', 'active_levera_pay']
for field in boolean_fields:
    if field in customers_parsed.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        value_counts = customers_parsed[field].value_counts(dropna=False)
        print(value_counts)

# Numeric fields analysis
numeric_fields = ['order_count', 'purchased_amount', 'reward_point', 'current_debts']
for field in numeric_fields:
    if field in customers_parsed.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        stats = customers_parsed[field].describe()
        print(f"Count: {stats['count']}")
        print(f"Mean: {stats['mean']:.2f}")
        print(f"Min: {stats['min']}")
        print(f"Max: {stats['max']}")
        print(f"Null count: {customers_parsed[field].isnull().sum()}")

# 4. Overall quality summary
print(f"\n4. OVERALL QUALITY SUMMARY")
total_columns = len(customers_parsed.columns)
high_null_columns = len(null_ratio[null_ratio > 0.7])
print(f"Total columns: {total_columns}")
print(f"Columns with >70% null: {high_null_columns}")
print(f"Duplicate customers: {dup_count}")
print(f"Data quality score: {((total_columns - high_null_columns) / total_columns * 100):.1f}%")


=== DATA QUALITY ANALYSIS ===

1. NULL RATIO ANALYSIS
Tỷ lệ null theo cột (sorted):
  email: 1.000 (100.0%)
  total_amount_referred: 1.000 (100.0%)
  address_postcode: 1.000 (100.0%)
  used_reward_point: 1.000 (100.0%)
  date_of_birth: 1.000 (100.0%)
  currency: 0.979 (97.9%)
  last_order_at: 0.261 (26.1%)
  conversation_link: 0.212 (21.2%)
  gender: 0.149 (14.9%)
  address_commune_id: 0.101 (10.1%)
  address_district_id: 0.101 (10.1%)
  address_province_id: 0.101 (10.1%)
  address_full: 0.101 (10.1%)
  fb_id: 0.004 (0.4%)
  phone: 0.002 (0.2%)
  order_sources: 0.001 (0.1%)
  name: 0.000 (0.0%)
  customer_id: 0.000 (0.0%)
  is_block: 0.000 (0.0%)
  is_discount_by_level: 0.000 (0.0%)
  inserted_at: 0.000 (0.0%)
  purchased_amount: 0.000 (0.0%)
  returned_order_count: 0.000 (0.0%)
  succeed_order_count: 0.000 (0.0%)
  order_count: 0.000 (0.0%)
  id: 0.000 (0.0%)
  shop_id: 0.000 (0.0%)
  updated_at: 0.000 (0.0%)
  referral_code: 0.000 (0.0%)
  active_levera_pay: 0.000 (0.0%)
  count_refe

## Bước 6: Data Cleaning & Transformation

### Mục tiêu
- Drop các cột có tỷ lệ null cao (>70%)
- Chuẩn hóa datetime fields
- Ép kiểu numeric cho các cột số
- Chuẩn hóa gender field về M/F/O
- Xử lý các giá trị bất thường

### Quy trình
1. **Drop high-null columns**: Tự động drop cột null > 70%
2. **Datetime conversion**: Chuyển đổi string sang datetime
3. **Numeric conversion**: Ép kiểu cho các cột số
4. **String normalization**: Chuẩn hóa gender và các trường string
5. **Validation**: Kiểm tra kết quả cleaning

### Decision Rules
- Drop columns với null > 70% (configurable threshold)
- Convert datetime với error handling (coerce)
- Convert numeric với error handling (coerce)
- Standardize gender: male/female → M/F, phi nhị giới → O


In [6]:
def drop_high_null_cols(df, threshold=0.7):
    """
    Xóa các cột có tỷ lệ null > threshold (mặc định 70%)
    """
    null_ratio = df.isnull().mean()
    cols_to_drop = null_ratio[null_ratio > threshold].index.tolist()
    
    print(f"Các cột bị drop (null > {threshold*100}%): {cols_to_drop}")
    
    return df.drop(columns=cols_to_drop, errors="ignore")

# Bắt đầu data cleaning
print("=== DATA CLEANING & TRANSFORMATION ===")

# 1. Drop high-null columns
print("\n1. DROPPING HIGH-NULL COLUMNS")
customers_clean = drop_high_null_cols(customers_parsed, threshold=0.7)
print(f"Shape after dropping: {customers_clean.shape}")

# 2. Datetime conversion
print("\n2. DATETIME CONVERSION")
datetime_cols = ["inserted_at", "updated_at", "last_order_at", "date_of_birth"]
for col in datetime_cols:
    if col in customers_clean.columns:
        print(f"Converting {col} to datetime...")
        customers_clean[col] = pd.to_datetime(customers_clean[col], errors="coerce")

# 3. Numeric conversion
print("\n3. NUMERIC CONVERSION")
numeric_cols = [
    "order_count", "succeed_order_count", "returned_order_count",
    "purchased_amount", "reward_point", "used_reward_point",
    "current_debts", "count_referrals", "total_amount_referred",
    "shop_id"
]
for col in numeric_cols:
    if col in customers_clean.columns:
        print(f"Converting {col} to numeric...")
        customers_clean[col] = pd.to_numeric(customers_clean[col], errors="coerce")

# 4. String normalization
print("\n4. STRING NORMALIZATION")

# Gender standardization
if "gender" in customers_clean.columns:
    print("Standardizing gender field...")
    customers_clean["gender"] = customers_clean["gender"].str.lower().replace({
        "male": "M", "female": "F",
        "nam": "M", "nữ": "F", 
        "phi nhị giới": "O", "nonbinary": "O"
    })

# Name standardization
if "name" in customers_clean.columns:
    print("Standardizing name field...")
    customers_clean["name"] = customers_clean["name"].str.strip().str.title()

# Phone standardization (remove non-numeric characters except comma)
if "phone" in customers_clean.columns:
    print("Standardizing phone field...")
    customers_clean["phone"] = customers_clean["phone"].str.replace(r'[^\d+,]', '', regex=True)

# 5. Validation
print("\n5. CLEANING VALIDATION")
print(f"Final shape: {customers_clean.shape}")
print(f"Final columns: {len(customers_clean.columns)}")
print(f"Memory usage: {customers_clean.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Check data types
print(f"\nData types summary:")
print(customers_clean.dtypes.value_counts())

# Sample cleaned data
print(f"\n=== SAMPLE CLEANED DATA ===")
print(customers_clean.head(3))


=== DATA CLEANING & TRANSFORMATION ===

1. DROPPING HIGH-NULL COLUMNS
Các cột bị drop (null > 70.0%): ['email', 'date_of_birth', 'used_reward_point', 'total_amount_referred', 'address_postcode', 'currency']
Shape after dropping: (36090, 27)

2. DATETIME CONVERSION
Converting inserted_at to datetime...
Converting updated_at to datetime...
Converting last_order_at to datetime...

3. NUMERIC CONVERSION
Converting order_count to numeric...
Converting succeed_order_count to numeric...
Converting returned_order_count to numeric...
Converting purchased_amount to numeric...
Converting reward_point to numeric...
Converting current_debts to numeric...
Converting count_referrals to numeric...
Converting shop_id to numeric...

4. STRING NORMALIZATION
Standardizing gender field...
Standardizing name field...
Standardizing phone field...

5. CLEANING VALIDATION
Final shape: (36090, 27)
Final columns: 27
Memory usage: 37.33 MB

Data types summary:
object            13
int64              8
datetime64[

## Bước 7: Load vào Silver Database

### Mục tiêu
- Định nghĩa explicit schema mapping cho từng cột
- Load dữ liệu đã clean vào Silver database
- Tạo bảng dim_customers với schema tối ưu
- Xác nhận load thành công

### Quy trình
1. **Schema definition**: Định nghĩa dtype mapping cho từng cột
2. **Database load**: Sử dụng to_sql với explicit schema
3. **Validation**: Kiểm tra số record load thành công
4. **Schema verification**: Xác nhận schema trong database

### Schema Strategy
- VARCHAR(100): Cho các ID fields
- VARCHAR(255): Cho name fields
- VARCHAR(500): Cho URL/link fields  
- Text(): Cho các field có thể dài
- BigInteger(): Cho shop_id và các số lớn
- DateTime(): Cho timestamp fields
- Boolean(): Cho các trường boolean

> **Decision Point**: Xác nhận số record load thành công và kiểm tra schema ở Silver có đúng với kỳ vọng.


In [7]:
from sqlalchemy.types import BigInteger, Integer, Float, DateTime, Text, VARCHAR, Boolean

# Định nghĩa schema mapping cho từng cột
print("=== SCHEMA DEFINITION ===")

dtype_mapping = {}

# ID fields - VARCHAR(100)
id_fields = ["customer_id", "id", "fb_id"]
for field in id_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(100)

# Name fields - VARCHAR(255)  
name_fields = ["name", "referral_code"]
for field in name_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(255)

# URL/Link fields - VARCHAR(500)
url_fields = ["conversation_link"]
for field in url_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(500)

# Text fields - Text()
text_fields = ["phone", "email", "order_sources", "address_full"]
for field in text_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Text()

# Numeric fields
if "shop_id" in customers_clean.columns:
    dtype_mapping["shop_id"] = BigInteger()

# Integer fields
integer_fields = ["order_count", "succeed_order_count", "returned_order_count", 
                  "reward_point", "used_reward_point", "current_debts", 
                  "count_referrals", "total_amount_referred"]
for field in integer_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Integer()

# Float fields
float_fields = ["purchased_amount"]
for field in float_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Float()

# DateTime fields
datetime_fields = ["inserted_at", "updated_at", "last_order_at", "date_of_birth"]
for field in datetime_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = DateTime()

# Boolean fields
boolean_fields = ["is_block", "is_discount_by_level", "active_levera_pay"]
for field in boolean_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Boolean()

# String fields (default)
string_fields = ["gender", "currency", "address_province_id", "address_district_id", 
                "address_commune_id", "address_postcode"]
for field in string_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(50)

print(f"Schema mapping defined for {len(dtype_mapping)} columns")
print("Schema mapping:")
for col, dtype in dtype_mapping.items():
    print(f"  {col}: {dtype}")

# Load vào Silver database
print(f"\n=== LOADING TO SILVER DATABASE ===")
table_name = "dim_customers"

try:
    customers_clean.to_sql(
        table_name,
        con=silver_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"✅ Đã load {customers_clean.shape[0]} records vào Silver: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, silver_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"✅ Verification: {loaded_count} records trong database")
    print(f"✅ Schema: {len(dtype_mapping)} columns với explicit typing")
    
except Exception as e:
    print(f"❌ Error loading to Silver: {str(e)}")
    raise


=== SCHEMA DEFINITION ===
Schema mapping defined for 27 columns
Schema mapping:
  customer_id: VARCHAR(100)
  id: VARCHAR(100)
  fb_id: VARCHAR(100)
  name: VARCHAR(255)
  referral_code: VARCHAR(255)
  conversation_link: VARCHAR(500)
  phone: TEXT
  order_sources: TEXT
  address_full: TEXT
  shop_id: BIGINT
  order_count: INTEGER
  succeed_order_count: INTEGER
  returned_order_count: INTEGER
  reward_point: INTEGER
  current_debts: INTEGER
  count_referrals: INTEGER
  purchased_amount: FLOAT
  inserted_at: DATETIME
  updated_at: DATETIME
  last_order_at: DATETIME
  is_block: BOOLEAN
  is_discount_by_level: BOOLEAN
  active_levera_pay: BOOLEAN
  gender: VARCHAR(50)
  address_province_id: VARCHAR(50)
  address_district_id: VARCHAR(50)
  address_commune_id: VARCHAR(50)

=== LOADING TO SILVER DATABASE ===


✅ Đã load 36090 records vào Silver: dim_customers
✅ Verification: 36090 records trong database
✅ Schema: 27 columns với explicit typing


## Bước 8: Data Dictionary Generation

### Mục tiêu
- Tạo Data Dictionary chi tiết cho bảng dim_customers
- Document metadata của từng cột (data type, null %, unique count, sample values)
- Lưu Data Dictionary vào Excel file để tham khảo
- Tạo tài liệu tham chiếu cho Gold layer design

### Quy trình
1. **Generate dictionary**: Tạo DataFrame với metadata của từng cột
2. **Add business context**: Thêm business meaning cho các trường
3. **Export to Excel**: Lưu vào file Excel để documentation
4. **Summary report**: Tạo báo cáo tổng kết

### Data Dictionary Fields
- column: Tên cột
- dtype: Kiểu dữ liệu (pandas)
- sql_type: Kiểu dữ liệu SQL
- null_pct: Tỷ lệ null (%)
- unique_count: Số lượng giá trị unique
- sample_values: Các giá trị mẫu
- business_meaning: Ý nghĩa business
- extraction_date: Ngày extract

> **Decision Point**: Dựa vào Data Dictionary, kiểm tra lại schema Silver có phù hợp với Business Requirement. Nếu thiếu cột quan trọng hoặc dtype chưa chuẩn, cần quay lại bước 4-6 để chỉnh sửa.


In [ ]:
# Generate Data Dictionary
print("=== GENERATING DATA DICTIONARY ===")

def get_business_meaning(column_name):
    """Get business meaning for each column"""
    business_meanings = {
        # Định danh & Cơ bản
        "customer_id": "Unique customer identifier (primary key)",
        "id": "Internal system ID for customer",
        "name": "Customer full name",
        "shop_id": "Shop identifier (foreign key to dim_shops)",
        
        # Thông tin cá nhân
        "gender": "Customer gender (M/F/O)",
        "phone": "Customer phone numbers (comma-separated)",
        "email": "Customer email addresses (comma-separated)",
        "date_of_birth": "Customer date of birth",
        
        # Hành vi mua hàng
        "order_count": "Total number of orders placed by customer",
        "succeed_order_count": "Number of successful orders",
        "returned_order_count": "Number of returned orders",
        "purchased_amount": "Total amount spent by customer",
        "last_order_at": "Date of last order placed",
        
        # Thời gian
        "inserted_at": "Record creation timestamp",
        "updated_at": "Record last update timestamp",
        
        # Trạng thái
        "is_block": "Customer blocked status (true/false)",
        "is_discount_by_level": "Discount by level enabled (true/false)",
        "active_levera_pay": "Levera payment active (true/false)",
        
        # Tài chính & Loyalty
        "reward_point": "Current reward points balance",
        "used_reward_point": "Total reward points used",
        "current_debts": "Current outstanding debt amount",
        "count_referrals": "Number of successful referrals",
        "total_amount_referred": "Total amount from referrals",
        
        # Marketing & Tracking
        "referral_code": "Customer referral code",
        "fb_id": "Facebook ID for social tracking",
        "order_sources": "Order sources (comma-separated)",
        "conversation_link": "Link to customer conversation",
        
        # Địa chỉ
        "address_province_id": "Province ID from address",
        "address_district_id": "District ID from address",
        "address_commune_id": "Commune ID from address",
        "address_full": "Full address text",
        "address_postcode": "Postal code",
        
        # Metadata
        "currency": "Default currency for customer"
    }
    return business_meanings.get(column_name, "No business meaning defined")

# Tạo Data Dictionary
dict_data = []
for col in customers_clean.columns:
    col_info = {
        "table_name": "dim_customers",
        "column_name": col,
        "dtype": str(customers_clean[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": customers_clean[col].isnull().sum(),
        "null_pct": round(customers_clean[col].isnull().mean() * 100, 2),
        "unique_count": customers_clean[col].nunique(),
        "sample_values": str(customers_clean[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Append Data Dictionary vào file Excel
excel_path = "Technical_Document/Dictionary.xlsx"
try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Data_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== TRANSFORMATION SUMMARY ===")
print(f"Source records: {len(customers_df)}")
print(f"Target records: {len(customers_clean)}")
print(f"Columns extracted: {len(customers_clean.columns)}")
print(f"Columns dropped: {len(customers_parsed.columns) - len(customers_clean.columns)}")
print(f"Target table: Silver.dim_customers")
print(f"Data Dictionary: {excel_path}")
print(f"Transformation completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


=== GENERATING DATA DICTIONARY ===
Generated Data Dictionary for 27 columns

=== DATA DICTIONARY ===
       table_name           column_name           dtype      sql_type  \
0   dim_customers           customer_id          object  VARCHAR(100)   
1   dim_customers                    id          object  VARCHAR(100)   
2   dim_customers                  name          object  VARCHAR(255)   
3   dim_customers               shop_id           int64        BIGINT   
4   dim_customers                gender          object   VARCHAR(50)   
5   dim_customers                 phone          object          TEXT   
6   dim_customers           order_count           int64       INTEGER   
7   dim_customers   succeed_order_count           int64       INTEGER   
8   dim_customers  returned_order_count           int64       INTEGER   
9   dim_customers      purchased_amount           int64         FLOAT   
10  dim_customers         last_order_at  datetime64[ns]      DATETIME   
11  dim_customers      